In [1]:
# Importando los módulos que necesitaremos
import numpy as np

import pandas as pd
from pandas.plotting import table

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

import seaborn as sns

from datetime import datetime

import os

# Configurando los estílos de los gráficos
plt.ioff()
sns.set_context('talk')
sns.set_style("whitegrid")

# Definiendo Constantes
BINS = [0, 10, 20, 30, 40, 50, 60, 70, 80, 300]

## Limpieza de Datos

In [2]:
# Leo la fuente de datos local en CSV 
df = pd.read_csv('data/vacunas_covid.csv')
df.head()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO
0,20210303,c7ba6f9424c59a543e8803495c1061db,PERSONAL DE SALUD,39.0,MASCULINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO
1,20210303,51ffbe83478f1e64da17fe2756a9594b,PERSONAL DE SALUD,37.0,FEMENINO,20210219,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
2,20210303,54048fe3b06c35f14350d7e81340bdbd,PERSONAL DE SALUD,44.0,MASCULINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
3,20210303,746ad64c513b5d53b987ed48a6b45f77,PERSONAL DE SALUD,54.0,FEMENINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
4,20210303,80f0a41dd63af3b96d4a80bfab2eaa61,PERSONAL DE SALUD,35.0,FEMENINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO


In [3]:
# Obtengo el conteo del número de valores en cada columnas
df.count()

FECHA_CORTE         314197
UUID                314197
GRUPO_RIESGO        314197
EDAD                314020
SEXO                312945
FECHA_VACUNACION    314197
DOSIS               314197
FABRICANTE          314197
DIRESA              314197
DEPARTAMENTO        314197
PROVINCIA           314197
DISTRITO            314197
dtype: int64

In [4]:
# Verificamos que las columnas DEPARTAMENTO, GRUPO_RIESGO y SEXO tengan los valores indicados
print(df['DEPARTAMENTO'].unique())
print(df['GRUPO_RIESGO'].unique())
print(df['SEXO'].unique())

['LAMBAYEQUE' 'LIMA' 'AMAZONAS' 'LA LIBERTAD' 'CAJAMARCA' 'UCAYALI'
 'AYACUCHO' 'APURIMAC' 'SAN MARTIN' 'PIURA' 'HUANUCO' 'LORETO' 'CALLAO'
 'CUSCO' 'ICA' 'HUANCAVELICA' 'AREQUIPA' 'JUNIN' 'MADRE DE DIOS' 'PUNO'
 'PASCO' 'ANCASH' 'MOQUEGUA' 'TACNA' 'TUMBES']
['PERSONAL DE SALUD' 'TRABAJADOR Ó PERSONAL DE LIMPIEZA'
 'PERSONAL DE SEGURIDAD' 'PERSONAL MILITAR Ó FF AA'
 'POLICIA NACIONAL DEL PERU' 'ESTUDIANTES DE CIENCIAS DE LA SALUD'
 'BRIGADISTAS' 'CRUZ ROJA']
['MASCULINO' 'FEMENINO' nan]


In [5]:
# Filtramos las filas que no tienen valor en las columnas EDAD o SEXO
df = df[df['EDAD'].notnull() & df['SEXO'].notnull()]

## Transformación de Datos

In [6]:
# Abreviando los grupos de riesgo
df['GRUPO_RIESGO'].replace('PERSONAL DE SALUD','P. SALUD', inplace=True)
df['GRUPO_RIESGO'].replace('TRABAJADOR Ó PERSONAL DE LIMPIEZA','T. LIMP.', inplace=True)
df['GRUPO_RIESGO'].replace('PERSONAL DE SEGURIDAD','P. SEG.', inplace=True)
df['GRUPO_RIESGO'].replace('PERSONAL MILITAR Ó FF AA','FF.AA.', inplace=True)
df['GRUPO_RIESGO'].replace('POLICIA NACIONAL DEL PERU','P.N.P.', inplace=True)
df['GRUPO_RIESGO'].replace('ESTUDIANTES DE CIENCIAS DE LA SALUD','E. SALUD', inplace=True)
df['GRUPO_RIESGO'].replace('BRIGADISTAS','BRIG.', inplace=True)

# Genero Serie de grupos de riesgo
grupos_riesgo = df['GRUPO_RIESGO'].unique()
grupos_riesgo_series = pd.Series(np.zeros_like(grupos_riesgo),grupos_riesgo)

# Agrupo el conjunto de datos por DEPARTAMENTO, SEXO y EDAD para el DataFrame
gb_age = df[["DEPARTAMENTO","SEXO",'EDAD']].groupby(['DEPARTAMENTO', 'SEXO', pd.cut(df['EDAD'], BINS, False)])

# Agrupo el conjunto de datos por DEPARTAMENTO, SEXO y GRUPO_RIESGO para el DataFrame
gb_gr = df[["DEPARTAMENTO","SEXO",'GRUPO_RIESGO']].groupby(['DEPARTAMENTO', 'SEXO', 'GRUPO_RIESGO'])

In [7]:
def get_data_to_plot(department=None):
    if(department is None):
        return df[['SEXO','EDAD','GRUPO_RIESGO','FECHA_VACUNACION']]
    else:
        return df[df['DEPARTAMENTO'] == department][['SEXO','EDAD','GRUPO_RIESGO','FECHA_VACUNACION']]

## Visualización de Datos

In [8]:
# Visualizo los datos agrupados por EDAD via DataFrame
gb_age.size().unstack()

EDAD                     [0, 10)  [10, 20)  [20, 30)  [30, 40)  [40, 50)  \
DEPARTAMENTO  SEXO                                                         
AMAZONAS      FEMENINO         0        10       679      1187       699   
              MASCULINO        0         7       344       641       468   
ANCASH        FEMENINO         0         1      1128      1979      1465   
              MASCULINO        0         8       444       971       773   
APURIMAC      FEMENINO         0         3       855      1476      1120   
              MASCULINO        0         4       319       733       704   
AREQUIPA      FEMENINO         0         4      1661      3129      2381   
              MASCULINO        0         4       575      1222      1034   
AYACUCHO      FEMENINO         0         4       939      1769      1351   
              MASCULINO        0         5       338       837       845   
CAJAMARCA     FEMENINO         0         2      1578      2539      1616   
              MASCULINO        0        11       705      1355       951   
CALLAO        FEMENINO         0        18      1428      3601      3008   
              MASCULINO        0        25       962      1813      1627   
CUSCO         FEMENINO         0        10      1112      2896      1945   
              MASCULINO        0         9       457      1167      1094   
HUANCAVELICA  FEMENINO         0         6       887      1276       728   
              MASCULINO        0         4       334       599       464   
HUANUCO       FEMENINO         0         4      1078      1824      1157   
              MASCULINO        0        11       420      1006       760   
ICA           FEMENINO         0         4      1293      1845      1267   
              MASCULINO        0         5       435       819       650   
JUNIN         FEMENINO         0         4      1996      2882      1691   
              MASCULINO        0         9       584      1199       823   
LA LIBERTAD   FEMENINO         0         4      1792      3271      2171   
              MASCULINO        0        10       818      1484      1220   
LAMBAYEQUE    FEMENINO         0         1      1148      2027      1547   
              MASCULINO        0         1       447       909       805   
LIMA          FEMENINO         0        62      9894     24084     19277   
              MASCULINO        0        84      5047     11631      9765   
LORETO        FEMENINO         0         4       587      1343       922   
              MASCULINO        0         8       440       932       643   
MADRE DE DIOS FEMENINO         0         4       180       368       266   
              MASCULINO        0         1        75       180       129   
MOQUEGUA      FEMENINO         0         3       436       749       646   
              MASCULINO        0         5       142       346       288   
PASCO         FEMENINO         0         1       445       658       488   
              MASCULINO        0         3       167       329       311   
PIURA         FEMENINO         0        10      1541      2788      2431   
              MASCULINO        0        13       750      1417      1309   
PUNO          FEMENINO         0         3       924      1793      1156   
              MASCULINO        0         0       306       763       656   
SAN MARTIN    FEMENINO         0        12       977      1708       849   
              MASCULINO        0        12       558      1050       595   
TACNA         FEMENINO         0         4       457       825       763   
              MASCULINO        0         7       240       373       360   
TUMBES        FEMENINO         0         0       337       551       461   
              MASCULINO        0         1       182       298       271   
UCAYALI       FEMENINO         0         6       646       975       700   
              MASCULINO        0         7       404       595       469   

EDAD                     [50, 60)  [60, 70)  [7

In [9]:
# Visualizo los datos agrupados por GRUPO_RIESGO via DataFrame
gb_gr.size().unstack()

GRUPO_RIESGO             BRIG.  CRUZ ROJA  E. SALUD  FF.AA.  P. SALUD  \
DEPARTAMENTO  SEXO                                                      
AMAZONAS      FEMENINO     NaN        NaN       NaN     9.0    2926.0   
              MASCULINO    NaN        NaN       NaN    16.0    1737.0   
ANCASH        FEMENINO     NaN        NaN      39.0     NaN    6078.0   
              MASCULINO    NaN        NaN      26.0     5.0    3034.0   
APURIMAC      FEMENINO     NaN        NaN       NaN     NaN    4082.0   
              MASCULINO    NaN        NaN       NaN     NaN    2335.0   
AREQUIPA      FEMENINO     NaN        NaN       NaN     NaN   10529.0   
              MASCULINO    NaN        NaN       NaN     NaN    4337.0   
AYACUCHO      FEMENINO     NaN        NaN       4.0     NaN    5023.0   
              MASCULINO    NaN        NaN       3.0     NaN    2782.0   
CAJAMARCA     FEMENINO     NaN        NaN      13.0     NaN    6639.0   
              MASCULINO    NaN        NaN      12.0     NaN    3659.0   
CALLAO        FEMENINO     NaN        NaN       NaN    48.0   11252.0   
              MASCULINO    NaN        NaN       NaN    19.0    6157.0   
CUSCO         FEMENINO     NaN        NaN       NaN     NaN    7857.0   
              MASCULINO    NaN        NaN       NaN     3.0    3814.0   
HUANCAVELICA  FEMENINO     NaN        NaN       NaN     NaN    3321.0   
              MASCULINO    NaN        NaN       NaN     3.0    1729.0   
HUANUCO       FEMENINO     NaN        NaN       1.0     NaN    4875.0   
              MASCULINO    NaN        NaN       3.0     NaN    2739.0   
ICA           FEMENINO     NaN        NaN      53.0     1.0    5919.0   
              MASCULINO    NaN        NaN      22.0     NaN    2752.0   
JUNIN         FEMENINO     NaN        NaN      87.0     8.0    8010.0   
              MASCULINO    NaN        NaN      24.0    21.0    3350.0   
LA LIBERTAD   FEMENINO     NaN        NaN      12.0     NaN    9287.0   
              MASCULINO    NaN        NaN       5.0     NaN    4668.0   
LAMBAYEQUE    FEMENINO     NaN        NaN       NaN    15.0    6197.0   
              MASCULINO    NaN        NaN       NaN     4.0    3025.0   
LIMA          FEMENINO     1.0        NaN      10.0    25.0   75763.0   
              MASCULINO    1.0        NaN       3.0    20.0   38436.0   
LORETO        FEMENINO     NaN        NaN       3.0     1.0    3547.0   
              MASCULINO    NaN        NaN       1.0     2.0    2612.0   
MADRE DE DIOS FEMENINO     NaN        NaN       NaN     NaN    1016.0   
              MASCULINO    NaN        NaN       NaN     NaN     526.0   
MOQUEGUA      FEMENINO     NaN        NaN       NaN     NaN    2336.0   
              MASCULINO    NaN        NaN       NaN     NaN    1133.0   
PASCO         FEMENINO     NaN        NaN       2.0     2.0    1955.0   
              MASCULINO    NaN        NaN       NaN     2.0    1059.0   
PIURA         FEMENINO     NaN        NaN       7.0     5.0    8528.0   
              MASCULINO    1.0        NaN      14.0     2.0    4869.0   
PUNO          FEMENINO     NaN        NaN       1.0     NaN    5070.0   
              MASCULINO    NaN        NaN       NaN     NaN    2636.0   
SAN MARTIN    FEMENINO     NaN        NaN       NaN     NaN    4174.0   
              MASCULINO    NaN        NaN       NaN     2.0    2747.0   
TACNA         FEMENINO     NaN        NaN       NaN     NaN    2921.0   
              MASCULINO    NaN        NaN       NaN     NaN    1554.0   
TUMBES        FEMENINO     NaN        1.0       NaN     NaN    1587.0   
              MASCULINO    NaN        NaN       NaN     NaN     911.0   
UCAYALI       FEMENINO     NaN        NaN       NaN     NaN    2834.0   
              MASCULINO    NaN        NaN       NaN     2.0    1902.0   

GRUPO_RIESGO             P. SEG.  P.N.P.  T. LIMP.  
DEPARTAMENTO  SEXO                                  
AMAZONAS      FEMENINO       6.0    16.0      66.0  
              MASCULINO     33.0     7.0    

In [10]:
yticks = [5,15,25,35,45,55,65,75,85]

def generate_plot(area, data, date_range, show_plot=False, save_plot=True):

    # Creamos la figura, los ejes y agregamos la atribución
    plt.clf()

    fig, axs = plt.subplots(2,2, figsize=(20,20))
    plt.figtext(0.1,0.95, area + ': DISTRIBUCIÓN DE VACUNADOS CONTRA COVID-19 (N=' + str(len(data)) + ')', ha='left', fontsize=24)
    plt.figtext(0.1,0.925, 'Rango de fechas de vacunación: del {} al {}'.format(f'{date_range[0]:%Y-%m-%d}',f'{date_range[1]:%Y-%m-%d}'), ha='left', fontsize=20, color='#999')
    plt.figtext(0.9,0.02, """Fuente: https://www.datosabiertos.gob.pe/dataset/vacunaci%C3%B3n-contra-covid-19-ministerio-de-salud-minsa
    https://malexandersalazar.github.io/, Moisés Alexander Salazar Vila, """ + f'{datetime.now():%Y-%m-%d}', ha='right')

    # 1. Graficamos distribución por GRUPO ETAREO
    xlim_max = 0

    ## 1.1. Graficamos el primer histograma
    N, bins, patches = axs[0][0].hist(data["EDAD"], BINS, orientation = 'horizontal')

    ### Colorearemos las barras en base el conteo relativo de cada agrupación
    xlim_max = N.max() * 1.1
    fracs = N / N.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    ### Recorremos las barras y actualizamos cada color respectivamente
    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    ### Personalizamos las ejes de nuestro gráfico
    axs[0][0].set_yticklabels([])
    axs[0][0].yaxis.set_tick_params(width=0)
    axs[0][0].set_yticks(yticks, minor=True)
    axs[0][0].set_yticklabels(['0-9','10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+'], minor=True)
    axs[0][0].set_ylim(0,90)
    axs[0][0].set_xlim(0,xlim_max)
    axs[0][0].set_ylabel('GRUPO ETÁREO (AÑOS)')

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(BINS)-1):
        axs[0][0].text(N[i], yticks[i]-1, str(round(N[i]*100/len(data),2)) + '%')

    ## 1.2. Graficamos el segundo histograma
    males = data[data['SEXO'] == 'MASCULINO']["EDAD"]
    females = data[data['SEXO'] == 'FEMENINO']["EDAD"]

    N, bins, patches = axs[0][1].hist([males, females], BINS, orientation = 'horizontal')
    
    ### Personalizamos las ejes de nuestro gráfico
    axs[0][1].set_yticklabels([])
    axs[0][1].yaxis.set_tick_params(width=0)
    axs[0][1].set_yticks(yticks, minor=True)
    axs[0][1].set_ylim(0,90)
    axs[0][1].set_xlim(0,xlim_max)

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(BINS)-1):
        axs[0][1].text(N[0][i], yticks[i]-3, str(round(N[0][i]*100/len(data),2)) + '%')
        axs[0][1].text(N[1][i], yticks[i]+1, str(round(N[1][i]*100/len(data),2)) + '%')

    axs[0][1].legend(['Hombres (n=' + str(len(males)) + ')', 'Mujeres (n=' + str(len(females)) + ')'])

    # 2. Graficamos distribución por GRUPO RIESGO
    xlim_max = 0

    ## 2.1. Graficamos el primer gráfico de barras 
    group_counts = data['GRUPO_RIESGO'].value_counts().add(grupos_riesgo_series,fill_value=0)
    group_counts.sort_index(inplace=True,ascending=False)
    pos = np.array(range(len(group_counts)))
    patches = axs[1][0].barh(pos, group_counts.values, align='center')
    
    ### Colorearemos las barras en base el conteo relativo de cada agrupación
    xlim_max = group_counts.values.max() * 1.1
    fracs = group_counts.values / group_counts.values.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    ### Recorremos las barras y actualizamos cada color respectivamente
    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    ### Personalizamos las ejes de nuestro gráfico
    axs[1][0].set_yticks(pos)
    axs[1][0].set_yticklabels(list(group_counts.index.values))
    axs[1][0].set_xlim(0,xlim_max)

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(pos)):
        axs[1][0].text(group_counts.values[i], pos[i]-0.1, str(round(group_counts[i]*100/len(data),2)) + '%')

    ## 2.2. Graficamos el segundo gráfico de barras 
    males = data[data['SEXO'] == 'MASCULINO']
    males_group_counts = males["GRUPO_RIESGO"].value_counts().add(grupos_riesgo_series,fill_value=0)
    males_group_counts.sort_index(inplace=True,ascending=False)

    females = data[data['SEXO'] == 'FEMENINO']
    females_group_counts = females["GRUPO_RIESGO"].value_counts().add(grupos_riesgo_series,fill_value=0)
    females_group_counts.sort_index(inplace=True,ascending=False)

    pos = np.array(range(len(males_group_counts)))

    axs[1][1].barh(pos, males_group_counts.values, 0.35, align='center')
    axs[1][1].barh(pos+0.35, females_group_counts.values, 0.35, align='center')
    
    ### Personalizamos las ejes de nuestro gráfico
    axs[1][1].set_yticks(pos)
    axs[1][1].set_yticklabels([])
    axs[1][1].set_xlim(0,xlim_max)
    
    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(pos)):
        axs[1][1].text(males_group_counts.values[i], pos[i]-0.1, str(round(males_group_counts[i]*100/len(data),2)) + '%')
        axs[1][1].text(females_group_counts.values[i], pos[i]+0.3, str(round(females_group_counts[i]*100/len(data),2)) + '%')

    ## 3. Mostramos, guardamos y generamos el Markdown para las imágenes

    sns.despine(left=False, bottom=False)

    if save_plot:
        filename = 'dist/{}_{}.png'.format(f'{datetime.now():%Y%m%d}', area.replace(' ', '_'))
        plt.savefig("../" + filename, bbox_inches='tight')
        with open("../dist/images.txt", "a", encoding='utf-8') as f:
            f.write('![alt text]({} "{}")\n'.format(filename, area))

    if show_plot:
        plt.show()

In [11]:
# Eliminando archivo generado para el Markdown de las imágenes
if os.path.exists("../dist/images.txt"):
    os.remove("../dist/images.txt")

In [12]:
# Perú
data = get_data_to_plot()
dates = pd.to_datetime(data['FECHA_VACUNACION'], infer_datetime_format=False, format='%Y%m%d')
generate_plot('PERÚ', data, (dates.min(), dates.max()), False, True)

In [13]:
# Deparmanetos
departments_ordered = np.sort(df['DEPARTAMENTO'].unique())
for d in departments_ordered:
    data = get_data_to_plot(d)
    dates = pd.to_datetime(data['FECHA_VACUNACION'], infer_datetime_format=False, format='%Y%m%d')
    generate_plot(d, data, (dates.min(), dates.max()), False, True)